In [47]:
import codecs
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from scipy.spatial import distance

Load data in Tensorflow.

In [6]:
root = "/Users/pasquale/git/recommender/"
training_data_folder = [
    '%straining_data/web-radio/output/list' % root,
    '%straining_data/spotify/output/playlists/list' % root,
    '%straining_data/concerts/output/list/philharmonie' % root,
    '%straining_data/concerts/output/list/itema3' % root
    ]
embDir = '/Users/pasquale/git/music-embeddings'
what = 'artist'

uri_file = '%s/%s.emb.u' % (embDir, what)
vector_file = '%s/%s.emb.v' % (embDir, what)
header_file = '%s/%s.emb.h' % (embDir, what)
label_file = '%s/%s.emb.l' % (embDir, what)

In [7]:
# load embeddings
vectors = np.array([line.strip().split(' ') for line in codecs.open(vector_file, 'r', 'utf-8')])
heads = np.array([line.strip() for line in codecs.open(header_file, 'r', 'utf-8')])
uris = np.array([line.strip() for line in codecs.open(uri_file, 'r', 'utf-8')])
lbs = np.array([line.strip() for line in codecs.open(label_file, 'r', 'utf-8')])

In [8]:
head_label = heads[0].split()
head_val = heads[1].split()
head_dim = []
for i in range(0, len(head_val)):
    print(head_label[i] + ' -> ' + head_val[i])
    for j in range(0, int(head_val[i])):
        head_dim.append(head_label[i])
        
# head_dim.append('fake')
# head_dim.append('fake')
# head_dim.append('fake')
# head_dim

mop -> 80
birth_date -> 1
death_date -> 1
casting -> 80
function -> 100
genre -> 100
key -> 100


In [9]:
# load training
train_array = []
for folder in training_data_folder:
    training_folder = '%s/%s' % (folder, what)
    for f in os.listdir(training_folder):
        file= '%s/%s' % (training_folder, f)
        train_array.append(np.array([line.strip() for line in codecs.open(file, 'r', 'utf-8')]))

a= [t.replace('data.','overture.') for t in train_array[0]]

In [10]:
for b in a:
    print(b)

http://overture.doremus.org/artist/03954109-0253-35d6-a70e-89ab27dea09c
http://overture.doremus.org/artist/bcb7cc8c-7784-3d08-a075-96fd50a3f4f8
http://overture.doremus.org/artist/4802a043-23bb-3b8d-a443-4a3bd22ccc63
http://overture.doremus.org/artist/16c2e858-d298-3584-9c5f-49e02cebdc44
http://overture.doremus.org/artist/269cec9d-5025-3a8a-b2ef-4f7acb088f2b
http://overture.doremus.org/artist/d07ec69a-da7c-36ab-a098-44c0cdc9741d
http://overture.doremus.org/artist/2f800bb2-8979-39e3-b430-ea8aeac41190
http://overture.doremus.org/artist/821f1dd5-3139-3a98-9fa8-84176680cd9c
http://overture.doremus.org/artist/0607b9fe-c6fe-38c1-aa5a-c2cf957bee74
http://overture.doremus.org/artist/f9a2ac39-a62d-3be2-8abb-e564de0ec96d
http://overture.doremus.org/artist/0607b9fe-c6fe-38c1-aa5a-c2cf957bee74
http://overture.doremus.org/artist/a3e06c5c-4d9d-323a-a3e9-5121acb70d3e
http://overture.doremus.org/artist/34943da5-d068-3ada-bd4d-3db0c66c8a7d
http://overture.doremus.org/artist/03954109-0253-35d6-a70e-89ab2

Data pre-processing

In [11]:
negVector = -2. * np.ones_like(vectors[0], dtype=np.float32)

def get_embs(x):
    # uri to embedding
    v = vectors[np.argwhere(uris == x)]
    if v.size == 0:
        print(x)
        result = np.array(negVector)
    else:
        result = np.array(v[0][0])
    return result.astype(np.float32)

In [12]:
minibatches_len = 8

In [13]:
training_vector_all = []
for t in train_array:
    if(len(t) < minibatches_len):
        continue
    
    training_vector_all.append(np.array([get_embs(xi) for xi in t]))

http://data.doremus.org/artist/bcb7cc8c-7784-3d08-a075-96fd50a3f4f8
http://data.doremus.org/artist/a3e06c5c-4d9d-323a-a3e9-5121acb70d3e
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/62348fad-974d-38db-a5a7-505f6a7a7db6
http://data.doremus.org/artist/ce72d043-0acc-3cca-95ba-959903770bc8
http://data.doremus.org/artist/4bb4caff-600b-322b-9b8b-ef0d41d4a61f
http://data.doremus.org/artist/d1fc38b7-c0d6-374b-8b9a-083b185e2239
http://data.doremus.org/artist/626219bf-3b70-3cde-8f07-4d56337fc16a
http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/bcb7cc8c-7784-3d08-a075-96fd50a3f4f8
http://data.doremus.org/artist/13891f70-8df4-3345-827d-7824629d3f32
http://data.doremus.org/artist/08227d5f-2048-314d-9e05-ae14c28ec8cb
http://data.doremus.org/artist/cfb1b272-3036-3e6

http://data.doremus.org/artist/a3e06c5c-4d9d-323a-a3e9-5121acb70d3e
http://data.doremus.org/artist/47e04849-715e-3773-a2d1-a2ce2c7d320d
http://data.doremus.org/artist/4385ce9c-1fe2-3b59-9df0-9ac506d9ab43
http://data.doremus.org/artist/47e04849-715e-3773-a2d1-a2ce2c7d320d
http://data.doremus.org/artist/a3e06c5c-4d9d-323a-a3e9-5121acb70d3e
http://data.doremus.org/artist/f6b78374-45d2-33d0-b692-cc58d799edfb
http://data.doremus.org/artist/40e88279-dc12-3dd0-8aeb-3b8dd41af302
http://data.doremus.org/artist/0b448b47-7bb5-36cd-843d-c2a4e3b5cd83
http://data.doremus.org/artist/91b2943d-5947-3c99-a9aa-0157ed548ceb
http://data.doremus.org/artist/3ed4f326-18a2-3cc1-bc8b-3d5d41d0c1dd
http://data.doremus.org/artist/de475aff-c666-3c12-b69f-8d1b1a8d3473
http://data.doremus.org/artist/2af6f9cb-b497-3604-8dee-a264ddeda5cc
http://data.doremus.org/artist/05ba9cb0-1cf7-3dd9-802e-b4e7ff605f35
http://data.doremus.org/artist/91241c0b-2878-3b5d-8a6d-36289c12f287
http://data.doremus.org/artist/08227d5f-2048-314

http://data.doremus.org/artist/91241c0b-2878-3b5d-8a6d-36289c12f287
http://data.doremus.org/artist/2019b0fb-aec4-384f-aba7-754831a27078
http://data.doremus.org/artist/66a8f411-f232-37d4-89c1-27ebff74d0a2
http://data.doremus.org/artist/c51a1b01-5682-33da-a15a-67d7f36c8308
http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/6447cde9-fbd7-3da3-a1bf-ff1176456cd7
http://data.doremus.org/artist/499c5737-529f-32a1-9284-7d99b73b1be3
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/62348fad-974d-38db-a5a7-505f6a7a7db6
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/499c5737-529f-32a1-9284-7d99b73b1be3
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/33e555e9-f60d-3e6d-9afc-f27dca8596ce
http://data.doremus.org/artist/4bb4caff-600b-322b-9b8b-ef0d41d4a61f
http://data.doremus.org/artist/35ddad40-c793-312

http://data.doremus.org/artist/561a57a4-bd26-391b-b3c1-dd67e15e9469
http://data.doremus.org/artist/9c20ed07-6199-34da-af68-a29914f324b0
http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/bcb7cc8c-7784-3d08-a075-96fd50a3f4f8
http://data.doremus.org/artist/23df9200-a2a7-3ba8-ab11-ec16bf781c41
http://data.doremus.org/artist/8d7cdfa3-7e61-3e29-90c8-92249e5541bf
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/47e04849-715e-3773-a2d1-a2ce2c7d320d
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/c25adc8e-1634-34e7-82fa-c8519d8c9244
http://data.doremus.org/artist/a3e06c5c-4d9d-323a-a3e9-5121acb70d3e
http://data.doremus.org/artist/bcb7cc8c-7784-3d08-a075-96fd50a3f4f8
http://data.doremus.org/artist/bcb7cc8c-7784-3d08-a075-96fd50a3f4f8
http://data.doremus.org/artist/4bb4caff-600b-322b-9b8b-ef0d41d4a61f
http://data.doremus.org/artist/47e04849-715e-377

http://data.doremus.org/artist/5e166265-3cbf-3b0c-b7f5-a09b82e3a279
http://data.doremus.org/artist/c51a1b01-5682-33da-a15a-67d7f36c8308
http://data.doremus.org/artist/47e04849-715e-3773-a2d1-a2ce2c7d320d
http://data.doremus.org/artist/9df34892-15d2-3abc-9772-0060081e004f
http://data.doremus.org/artist/47e04849-715e-3773-a2d1-a2ce2c7d320d
http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/1929320a-c4f9-36e5-890c-657126ebcdaf
http://data.doremus.org/artist/5e166265-3cbf-3b0c-b7f5-a09b82e3a279
http://data.doremus.org/artist/2aeb07c8-09ac-3785-a93b-40444fc4cbc5
http://data.doremus.org/artist/1929320a-c4f9-36e5-890c-657126ebcdaf
http://data.doremus.org/artist/6447cde9-fbd7-3da3-a1bf-ff1176456cd7
http://data.doremus.org/artist/064a15a4-b08d-3300-9b18-91ca4e79671c
http://data.doremus.org/artist/2a9944d1-ce7b-389f-8dbc-fe4e74bf237b
http://data.doremus.org/artist/4385ce9c-1fe2-3b59-9df0-9ac506d9ab43
http://data.doremus.org/artist/19085a8b-d38d-37a

...

http://data.doremus.org/artist/1929320a-c4f9-36e5-890c-657126ebcdaf
http://data.doremus.org/artist/6447cde9-fbd7-3da3-a1bf-ff1176456cd7
http://data.doremus.org/artist/a6302b4b-a8bf-3952-a8e2-0775068c3f5c
http://data.doremus.org/artist/62348fad-974d-38db-a5a7-505f6a7a7db6
http://data.doremus.org/artist/a6302b4b-a8bf-3952-a8e2-0775068c3f5c
http://data.doremus.org/artist/499c5737-529f-32a1-9284-7d99b73b1be3
http://data.doremus.org/artist/62348fad-974d-38db-a5a7-505f6a7a7db6
http://data.doremus.org/artist/23df9200-a2a7-3ba8-ab11-ec16bf781c41
http://data.doremus.org/artist/47e04849-715e-3773-a2d1-a2ce2c7d320d
http://data.doremus.org/artist/df6fc930-3a58-355d-8270-b2343b5f5b0a
http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/1929320a-c4f9-36e5-890c-657126ebcdaf
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/62348fad-974d-38db-a5a7-505f6a7a7db6
http://data.doremus.org/artist/35ddad40-c793-312

http://data.doremus.org/artist/6be5c3f6-d661-35d5-8b74-029d65729830
http://data.doremus.org/artist/2aeb07c8-09ac-3785-a93b-40444fc4cbc5
http://data.doremus.org/artist/9f5810ce-77f0-311b-8bcf-267f1ae61468
http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/df6fc930-3a58-355d-8270-b2343b5f5b0a
http://data.doremus.org/artist/62348fad-974d-38db-a5a7-505f6a7a7db6
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/6447cde9-fbd7-3da3-a1bf-ff1176456cd7
http://data.doremus.org/artist/dac301a0-0165-3bdb-9746-545ba575b127
http://data.doremus.org/artist/1f76d196-3e35-3bcc-bb34-86afba32c3a8
http://data.doremus.org/artist/62348fad-974d-38db-a5a7-505f6a7a7db6
http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/5e166265-3cbf-3b0c-b7f5-a09b82e3a279
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/1f76d196-3e35-3bc

http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/dac301a0-0165-3bdb-9746-545ba575b127
http://data.doremus.org/artist/62348fad-974d-38db-a5a7-505f6a7a7db6
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/a6302b4b-a8bf-3952-a8e2-0775068c3f5c
http://data.doremus.org/artist/ce72d043-0acc-3cca-95ba-959903770bc8
http://data.doremus.org/artist/47e04849-715e-3773-a2d1-a2ce2c7d320d
http://data.doremus.org/artist/dac301a0-0165-3bdb-9746-545ba575b127
http://data.doremus.org/artist/df6fc930-3a58-355d-8270-b2343b5f5b0a
http://data.doremus.org/artist/35ddad40-c793-312a-86ec-6a1310c030a1
http://data.doremus.org/artist/778332bf-c1a7-32b1-8c19-578385e1c6ce
http://data.doremus.org/artist/509b09ef-a4e5-3ad7-90d2-ef540c30617a
http://data.doremus.org/artist/1929320a-c4f9-36e5-890c-657126ebcdaf
http://data.doremus.org/artist/f1fb6fa7-767a-3067-a55b-5b41de0f0367
http://data.doremus.org/artist/4385ce9c-1fe2-3b5

http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/5b2f91fa-0b3e-3c8f-ab74-456cb2207d19
http://data.doremus.org/artist/0ea76b7f-9eca-35c3-8e32-8ba22b19a96e
http://data.doremus.org/artist/282a4b9b-c927-3afc-ab83-59779cdc9f4a
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b

http://data.doremus.org/artist/8e517e6d-2ba2-3e54-935c-4c1b13a296ee
http://data.doremus.org/artist/da56c261-8a8a-3beb-8b44-329c1834f33b
http://data.doremus.org/artist/5b2f91fa-0b3e-3c8f-ab74-456cb2207d19
http://data.doremus.org/artist/8c214179-d32f-33ed-ad3f-c90a9a33dda3
http://data.doremus.org/artist/c889819f-71d2-3a2b-832b-8ac4f5b9fe6c
http://data.doremus.org/artist/2a7c47cb-53ca-321f-81dd-07b7231fe522
http://data.doremus.org/artist/4cd4a932-e91f-350e-84d7-883ac39d11bd
http://data.doremus.org/artist/0ea76b7f-9eca-35c3-8e32-8ba22b19a96e
http://data.doremus.org/artist/d8e35bbd-f26b-3ccc-9780-663cf2873f08
http://data.doremus.org/artist/ee29a0fb-422c-38

http://data.doremus.org/artist/ee1e880e-edbb-3aa0-92f8-ee8a2f219aad
http://data.doremus.org/artist/6d038bcc-1972-3233-bdd9-10a88c064b81
http://data.doremus.org/artist/fbbb60c9-4fa6-397d-bfbc-7a2f64931d76
http://data.doremus.org/artist/4bf84e2a-7315-39c2-afce-b7313ab9a7b5
http://data.doremus.org/artist/53f6a9c7-abad-347d-9dc5-f8050ae45e0f
http://data.doremus.org/artist/de7a8c9c-c64b-3954-af5b-b2a551f503f3
http://data.doremus.org/artist/1af5a91b-5715-3ed1-bfaf-97960a2c834e
http://data.doremus.org/artist/c5fd1625-3502-329f-a575-475d3639c63f
http://data.doremus.org/artist/2a7c47cb-53ca-321f-81dd-07b7231fe522
http://data.doremus.org/artist/19085a8b-d38d-37ac-9f0a-b412d6e1f88b
http://data.doremus.org/artist/ff43237b-3ffc-36ea-806f-758191231b1a
http://data.doremus.org/artist/65842a4d-bb30-3495-b0f7-84edf9a4c96d
http://data.doremus.org/artist/ab79542d-065f-3e82-9c40-4166ffde61fc
http://data.doremus.org/artist/71d28762-fb73-3e72-b83a-1bc1a85de77c
http://data.doremus.org/artist/814fbfce-761b-397

I divide the playlists in minibatches of 2X songs (X seeds and X targets)

In [14]:
training_vector = []

for t in training_vector_all:
    for x in range(1, len(t) - minibatches_len):
        training_vector.append(t[0:minibatches_len])
        
training_vector = np.array(training_vector)
training_vector.shape

(4258, 8, 462)

In [15]:
# train_label[399]

In [16]:
def smart_mean(a):
    b = np.where(a < -1., 0., a)
    s = np.sum(b, axis=1)
    c = np.count_nonzero(b, axis=1)
    d = np.divide(s, c)
    return np.where(np.isnan(d), 0, d)

Split test and train

In [17]:
train, test = train_test_split(training_vector, train_size=0.7)

train_seed, train_target = np.split(train, 2, axis=1)
train_vector = smart_mean(train_seed)
train_label = smart_mean(train_target)

test_seed, test_target = np.split(test, 2, axis=1)
test_vector = smart_mean(test_seed)
test_label = smart_mean(test_target)

/Users/pasquale/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/Users/pasquale/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [18]:
print('Train')
print(train_vector.shape)
print(train_label.shape)
print('Test')
print(test_vector.shape)
print(test_label.shape)

Train
(2980, 462)
(2980, 462)
Test
(1278, 462)
(1278, 462)


In [19]:
# Parameters
learning_rate = 0.1
num_steps = 10000
batch_size = 128
display_step = 1000

In [20]:
# Network Parameters
n_hidden_1 = 256  # 1st layer number of neurons
n_hidden_2 = 256  # 2nd layer number of neurons
num_input = train_vector[0].size
num_output = train_vector[0].size

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input], name="X")
Y = tf.placeholder(tf.float32, [None, num_output], name="Y")
Y_exp = tf.placeholder(tf.float32, [None, minibatches_len, num_output], name="Y_exp")

Neural network

In [21]:
# Create model
def neural_net(x):
    with tf.name_scope('hidden_1') as scope:
        # Hidden fully connected layer with 256 neurons
        w1 = tf.Variable(tf.random_normal([num_input, n_hidden_1]), name='w')
        b1 = tf.Variable(tf.random_normal([n_hidden_1]), name='b')
        layer_1 = tf.add(tf.matmul(x, w1), b1, name='o')
        layer_1 = tf.nn.relu(layer_1, name='u')
        
    with tf.name_scope('hidden_2') as scope:
        # Hidden fully connected layer with 256 neurons
        w2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='w')
        b2 = tf.Variable(tf.random_normal([n_hidden_2]), name='b')
        layer_2 = tf.add(tf.matmul(layer_1, w2), b2, name='o')
        layer_2 = tf.nn.relu(layer_2, name='u')

    with tf.name_scope('out_layer') as scope:
        wo = tf.Variable(tf.random_normal([n_hidden_2, num_output]), name='w')
        bo = tf.Variable(tf.random_normal([num_output]), name='b')
        out_layer = tf.add(tf.matmul(layer_2, wo), bo, name="o")
    return tf.nn.l2_normalize(out_layer, axis=1, name = 'y')

In [35]:
def weighted_l2(a, b, w):
    with tf.name_scope('weighted_l2') as scope:
#         # https://stackoverflow.com/a/8861999/1218213
#         q = tf.subtract(a, b, name="q")
#         # return np.sqrt((w * q * q).sum())
#         pow_q = tf.cast(tf.pow(q, 2), tf.float32, name="q-power")

#         return tf.reduce_sum(tf.multiply(w, pow_q), axis=1, name="o", keepdims=True)
        normalize_a = tf.nn.l2_normalize(a,0)        
        normalize_b = tf.nn.l2_normalize(b,0)
        return tf.reduce_sum(tf.multiply(normalize_a,normalize_b, axis=1), axis=1, keepdims=True)
#         sess=tf.Session()
#         cos_sim=sess.run(cos_similarity,feed_dict={a:[1,2,3],b:[2,4,6]})


In [36]:
def compute_penalty(expected, taken, total):
    with tf.name_scope('penalty') as scope:
        penalty = tf.divide(tf.subtract(expected, taken), total)
        return tf.cast(penalty, tf.float32)

In [37]:
# Construct model
logits = neural_net(X)

In [38]:
logits.shape

TensorShape([Dimension(None), Dimension(462)])

In [48]:
def computeSimilarity(seed, target, w):
    b1 = np.where(seed < -1)[0]
    b2 = np.where(target < -1)[0]
    bad_pos = np.unique(np.concatenate([b1, b2]))

    _seed = np.delete(seed, bad_pos, axis=0)
    _target = np.delete(target, bad_pos, axis=0)
    _w = np.delete(w, bad_pos, axis=0)

    if len(_seed) == 0:
        return 0

    # distance
    d = weightedL2(_seed, _target, _w)

    # how much info I am not finding
    penalty = len([x for x in b2 if x not in b1]) / len(seed)

    # score
    s = (max_distance - d) / max_distance
    return s * (1 - penalty)


def weightedL2(a, b, w=1):
    return distance.cosine(a,b)
    # https://stackoverflow.com/a/8861999/1218213
    q = a - b
    # return np.sqrt((w * q * q).sum())
    return (w * q * q).sum()


def find(seed, n=4, w=None):
    global max_distance
    global vectors
    global uris
    global lbs
    
    f_length = len(seed)
    
    _seed = seed
    
    if w is None:
        w = np.ones(len(_seed))
        w = w / w.sum()
    else:
        w = np.array(w)
        temp = [np.ones(f_length[k]) * w[k] for k in range(len(w))]
        w = np.array([item for sublist in temp for item in sublist])

    max_distance = weightedL2(np.ones(len(_seed)), np.ones(len(_seed)) * -1, w)

    print('computing scores')
    scores = np.array([[computeSimilarity(_seed, x.astype(float), w) for x in vectors]])
    full = np.concatenate([uris.reshape(len(uris), 1), scores.transpose(), lbs.reshape(len(uris), 1)], axis=1)

    # remove the seed from the list
#     full = np.delete(full, pos, 0)

    # sort
    full_sorted = sorted(full, key=lambda _x: float(_x[1]), reverse=True)
    most_similar = full_sorted[:n]
    print('\n'.join('%s %s\n%s' % (f[0], f[1], f[2]) for f in most_similar))

    return [{'uri': _a[0], 'score': _a[1]} for _a in most_similar]

In [49]:
print(uris.shape)

(23156,)


In [50]:
# Define loss and optimizer
# loss_op = MSE
loss_op = tf.reduce_mean(tf.square(tf.subtract(logits, Y)), name='loss_op')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='optimizer')
train_op = optimizer.minimize(loss_op)
error_summary = tf.summary.scalar('error', loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(logits, Y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge([error_summary, accuracy_summary])

In [51]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [43]:
def next_batch(num, data, labels):
    """
    Return a total of `num` random samples and labels. 
    """
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]
    labels_shuffle = labels[idx]
    return data_shuffle, labels_shuffle

In [44]:
sess = tf.Session()
now = datetime.now()
log_dir = '%straining/output/%s' % (root, now.strftime("%Y%m%d-%H%M%S"))
writer = tf.summary.FileWriter(log_dir, sess.graph)

# Run the initializer
sess.run(init)

print("Start learning")
for step in range(1, num_steps + 1):
    batch_x, batch_y = next_batch(batch_size, train_vector, train_label)
#     print(batch_y)
    # Run optimization op (backprop)
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
    if step % display_step == 0 or step == 1:
#         print(batch_y[0])
        # Calculate batch loss and accuracy
        preds, summary, loss, acc = sess.run([logits, merged, loss_op, accuracy],
                                                feed_dict={X: batch_x, Y: batch_y})
        
        writer.add_summary(summary, step)

        print("Step " + str(step) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))
#         print("Predictions %s VS %s" % (preds[0][0], batch_y[0][0]))
        np.set_printoptions(precision=2)

print("Optimization Finished!")

print("Testing Accuracy:",
      sess.run(accuracy, feed_dict={X: test_vector, Y: test_label}))
writer.flush()
writer.close()



Start learning
Step 1, Minibatch Loss= 0.0067, Training Accuracy= 0.000
Step 1000, Minibatch Loss= 0.0022, Training Accuracy= 0.000
Step 2000, Minibatch Loss= 0.0022, Training Accuracy= 0.000
Step 3000, Minibatch Loss= 0.0022, Training Accuracy= 0.000
Step 4000, Minibatch Loss= 0.0022, Training Accuracy= 0.000
Step 5000, Minibatch Loss= 0.0023, Training Accuracy= 0.000
Step 6000, Minibatch Loss= 0.0021, Training Accuracy= 0.000
Step 7000, Minibatch Loss= 0.0022, Training Accuracy= 0.000
Step 8000, Minibatch Loss= 0.0022, Training Accuracy= 0.000
Step 9000, Minibatch Loss= 0.0022, Training Accuracy= 0.000
Step 10000, Minibatch Loss= 0.0022, Training Accuracy= 0.000
Optimization Finished!
Testing Accuracy: 0.0


In [45]:
verdi = get_embs('http://data.doremus.org/artist/b82c0771-5280-39af-ad2e-8ace2f4ebda3')
rossini = get_embs('http://data.doremus.org/artist/c5d5735c-1095-3ed4-a20f-1208ab9567f9')
rossini_pp = get_embs('http://data.doremus.org/artist/32c2b0ff-35f1-3e65-b0ca-34aaf35f3d50')
beethoven = get_embs('http://data.doremus.org/artist/6963af5e-b126-3d40-a84b-97e0b78f5452')
mozart = get_embs('http://data.doremus.org/artist/4802a043-23bb-3b8d-a443-4a3bd22ccc63')
ravel = get_embs('http://data.doremus.org/artist/1b1205f9-b99a-3bb8-ba77-256689af2e00')
gershwin = get_embs('http://data.doremus.org/artist/5b2ec204-a456-3aa2-8ac7-25305464add8')
coltrane = get_embs('http://data.doremus.org/artist/5425efed-002f-3638-a7b0-ad379a2bf63d')

In [53]:
preds = sess.run(logits, feed_dict={X: verdi.reshape(1,462), Y: test_label[0].reshape(1,462)})
print(preds)
nearest = find(preds[0])

[[ 6.60e-04  1.05e-03  2.64e-04  2.41e-03  2.74e-04 -1.24e-03  1.35e-03
   6.77e-04  2.06e-03  2.89e-04 -1.46e-03 -1.92e-03 -1.60e-03  3.89e-03
   1.10e-04 -1.05e-03 -5.48e-04 -9.02e-04  5.97e-05  1.10e-03  1.76e-03
   1.48e-03  2.17e-04  8.56e-04 -1.73e-03 -3.34e-04  2.48e-03 -7.35e-04
  -1.05e-03 -2.25e-03  1.79e-03  1.19e-03  6.42e-04 -1.29e-03 -2.61e-03
   2.25e-03 -1.47e-03 -6.53e-04  1.94e-03  6.40e-04 -1.46e-03 -9.78e-04
  -5.01e-04 -7.89e-04  4.47e-04 -2.89e-03 -9.98e-04  1.07e-03 -3.23e-03
   1.58e-03 -4.17e-04  5.60e-04 -3.89e-04  8.92e-04 -3.36e-04 -2.84e-03
   7.41e-04 -2.46e-04 -8.98e-04 -1.08e-03  9.27e-04 -2.12e-03  1.17e-03
  -6.55e-04 -1.21e-04  2.42e-03 -4.90e-03  6.15e-04 -2.84e-03 -8.44e-04
   1.93e-03  1.62e-03 -1.05e-03 -4.73e-04 -1.31e-03 -1.57e-04 -1.06e-03
  -8.28e-04  4.58e-03 -1.90e-03  3.88e-01  3.65e-01 -5.19e-04  2.37e-04
   2.45e-04 -1.28e-03 -5.59e-04 -1.49e-03  1.19e-03  5.81e-04  1.12e-03
   5.26e-04 -2.87e-04 -2.06e-05 -7.21e-04  6.72e-04  6.37e-04 -6

In [34]:
preds = sess.run(logits, feed_dict={X: verdi.reshape(1,342), Y: test_label[0].reshape(1,342)})
preds /= 1000
print(preds)

nearest = find(preds[0])


ValueError: cannot reshape array of size 462 into shape (1,342)